In [1]:
#| default_exp core.tensor
#| export

import numpy as np

# Constants for memory calculations
BYTES_PER_FLOAT32 = 4  # Standard float32 size in bytes
KB_TO_BYTES = 1024  # Kilobytes to bytes conversion
MB_TO_BYTES = 1024 * 1024  # Megabytes to bytes conversion

# Tensor class

In [2]:
#| export
class Tensor:
    """Tensor - the foundation of machine learning computation.

    This class provides the core data structure for all ML operations:
    - data: The actual numerical values (NumPy array)
    - shape: Dimensions of the tensor
    - size: Total number of elements
    - dtype: Data type (float32)

    All arithmetic, matrix, and shape operations are built on this foundation.
    """
    def __init__(self, data):
        """Create a new tensor from data."""
        self.data = np.array(data, dtype=np.float32)
        self.shape = self.data.shape
        self.size = self.data.size
        self.dtype = self.data.dtype

    def __repr__(self):
        "String representation for debugging"
        return f"Tensor(data={self.data}, shape={self.shape})"

    def __str__(self):
        "String representation"
        return f"Tensor({self.data})"

    def numpy(self):
        "Return the NumPy array"
        return self.data

    def memory_footprint(self):
        """Calculate exact memory usage in bytes.

        Systems Concept: Understanding memory footprint is fundamental to ML systems.
        Before running any operation, engineers should know how much memory it requires.

        Returns:
            int: Memory usage in bytes (e.g., 1000x1000 float32 = 4MB)
        """
        return self.data.nbytes

    def __add__(self, other):
        """Add two tensors element-wise with broadcasting support."""
        if isinstance(other, Tensor):
            return Tensor(self.data + other.data)
        return self.data + other

    def __sub__(self, other):
        """Subtract two tensors element-wise."""

        if isinstance(other, Tensor):
            return Tensor(self.data - other.data)
        return self.data - other

    def __mul__(self, other):
        """Multiply two tensors element-wise (NOT matrix multiplication)."""
        if isinstance(other, Tensor):
            return Tensor(self.data * other.data)
        return self.data * other

    def __truediv__(self, other):
        """Divide two tensors element-wise."""
        if isinstance(other, Tensor):
            return Tensor(self.data / other.data)
        return Tensor(self.data / other)

    def matmul(self, other):
        """Matrix multiplication of two tensors."""
        if not isinstance(other, Tensor):
            raise TypeError("Both elements must be tensors")
        if other.data.ndim == 0:
            return Tensor(self * other)
        if other.data.ndim == 1:
            return Tensor(np.matmul(self.data, other.data))
        if not self.shape[-1] == other.shape[-2]:
            raise ValueError(f"Inner dimensions must match. {self.shape[-1]} ≠ {other.shape[-2]}")
        if self.data.shape == (2,2):
            lst = []
            for i in range(2):
                for j in range(2):
                    result = np.dot(self.data[i,:],other.data[:,j])
                    lst.append(result)
            return Tensor(np.array(lst).reshape(2,2))
        return Tensor(np.matmul(self.data, other.data))

    def __matmul__(self, other):
        """Enable @ operator for matrix multiplication."""
        return self.matmul(other)

    def __getitem__(self, key):
        """Enable indexing and slicing operations on Tensors.

        TODO: Implement indexing and slicing that returns a new Tensor.

        APPROACH:
        1. Use NumPy indexing: self.data[key]
        2. If result is not an ndarray, wrap in np.array
        3. Return result wrapped in new Tensor

        EXAMPLE:
        >>> t = Tensor([[1, 2, 3], [4, 5, 6]])
        >>> row = t[0]  # First row
        >>> print(row.data)
        [1. 2. 3.]
        >>> element = t[0, 1]  # Single element
        >>> print(element.data)
        2.0

        HINT: NumPy's indexing already handles all complex cases (slicing, fancy indexing)
        """
        ### BEGIN SOLUTION


In [3]:
#################### TESTS
matrix = Tensor([[1, 2, 3], [4, 5, 6]])  # 2×3
vector = Tensor([1, 2, 3])  # 3×1 (conceptually)
result = matrix.matmul(vector)
result
# Expected: [1×1+2×2+3×3, 4×1+5×2+6×3] = [14, 32]
# expected = np.array([14, 32], dtype=np.float32)
# assert np.array_equal(result.data, expected)
# matrix = Tensor([[1, 2], [3, 4]])
# vector = Tensor([10, 20])
# result = matrix + vector
# expected = np.array([[11, 22], [13, 24]], dtype=np.float32)
# assert np.array_equal(result.data, expected)
# result
# matrix.data.ndim
# a = Tensor(2)
# a.data.ndim
# matrix.matmul(a)
#################### TESTS

Tensor(data=[14. 32.], shape=(2,))

# Unit tests

In [4]:
def test_unit_matrix_multiplication():
    """🧪 Test matrix multiplication operations."""
    print("🧪 Unit Test: Matrix Multiplication...")

    # Test 2×2 matrix multiplication (basic case)
    a = Tensor([[1, 2], [3, 4]])  # 2×2
    b = Tensor([[5, 6], [7, 8]])  # 2×2
    result = a.matmul(b)
    # Expected: [[1×5+2×7, 1×6+2×8], [3×5+4×7, 3×6+4×8]] = [[19, 22], [43, 50]]
    expected = np.array([[19, 22], [43, 50]], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test rectangular matrices (common in neural networks)
    c = Tensor([[1, 2, 3], [4, 5, 6]])  # 2×3 (like batch_size=2, features=3)
    d = Tensor([[7, 8], [9, 10], [11, 12]])  # 3×2 (like features=3, outputs=2)
    result = c.matmul(d)
    # Expected: [[1×7+2×9+3×11, 1×8+2×10+3×12], [4×7+5×9+6×11, 4×8+5×10+6×12]]
    expected = np.array([[58, 64], [139, 154]], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test matrix-vector multiplication (common in forward pass)
    matrix = Tensor([[1, 2, 3], [4, 5, 6]])  # 2×3
    vector = Tensor([1, 2, 3])  # 3×1 (conceptually)
    result = matrix.matmul(vector)
    # Expected: [1×1+2×2+3×3, 4×1+5×2+6×3] = [14, 32]
    expected = np.array([14, 32], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test shape validation - should raise clear error
    try:
        incompatible_a = Tensor([[1, 2]])     # 1×2
        incompatible_b = Tensor([[1], [2], [3]])  # 3×1
        incompatible_a.matmul(incompatible_b)  # 1×2 @ 3×1 should fail (2 ≠ 3)
        assert False, "Should have raised ValueError for incompatible shapes"
    except ValueError as e:
        assert "Inner dimensions must match" in str(e)
        assert "2 ≠ 3" in str(e)  # Should show specific dimensions

    print("✅ Matrix multiplication works correctly!")

if __name__ == "__main__":
    test_unit_matrix_multiplication()

🧪 Unit Test: Matrix Multiplication...
✅ Matrix multiplication works correctly!


In [5]:
def test_unit_arithmetic_operations():
    """🧪 Test arithmetic operations with broadcasting."""
    print("🧪 Unit Test: Arithmetic Operations...")

    # Test tensor + tensor
    a = Tensor([1, 2, 3])
    b = Tensor([4, 5, 6])
    result = a + b
    assert np.array_equal(result.data, np.array([5, 7, 9], dtype=np.float32))

    # Test tensor + scalar (very common in ML)
    result = a + 10
    assert np.array_equal(result.data, np.array([11, 12, 13], dtype=np.float32))

    # Test broadcasting with different shapes (matrix + vector)
    matrix = Tensor([[1, 2], [3, 4]])
    vector = Tensor([10, 20])
    result = matrix + vector
    expected = np.array([[11, 22], [13, 24]], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test subtraction (data centering)
    result = b - a
    assert np.array_equal(result.data, np.array([3, 3, 3], dtype=np.float32))

    # Test multiplication (scaling)
    result = a * 2
    assert np.array_equal(result.data, np.array([2, 4, 6], dtype=np.float32))

    # Test division (normalization)
    result = b / 2
    assert np.array_equal(result.data, np.array([2.0, 2.5, 3.0], dtype=np.float32))

    # Test chaining operations (common in ML pipelines)
    normalized = (a - 2) / 2  # Center and scale
    expected = np.array([-0.5, 0.0, 0.5], dtype=np.float32)
    assert np.allclose(normalized.data, expected)

    print("✅ Arithmetic operations work correctly!")

if __name__ == "__main__":
    test_unit_arithmetic_operations()

🧪 Unit Test: Arithmetic Operations...
✅ Arithmetic operations work correctly!


# Experiments